<a href="https://colab.research.google.com/github/AlirezaPNouri/Storytelling/blob/main/FinalExperimentalForPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The final version of storytelling algorithm to generate the experimental results for paper. 5/6/2022

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.0 MB/s 
     |████████████████████████████████| 77 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
     |████████████████████████████████| 880 kB 61.7 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ef909d42ef6af204862006aa5265cf8224164a53a63627f8cccf5eb9993badf9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [65]:
# needed packages
import requests
import numpy as np
import pandas as pd
import string
import pickle
import os
from numpy import median, array, exp


from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [3]:
#### Load the pre-traiend BERT model #####################
##########################################################
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
#constant variables
dataset_covid_link = 'https://raw.githubusercontent.com/AlirezaPNouri/Storytelling/main/datasets/short_parsed_covid_data.csv'
dataset_NYT_link = 'https://raw.githubusercontent.com/AlirezaPNouri/Storytelling/main/datasets/short_parsed_NYT_data.csv'
dataset_percentage = .6
MAX_DATA = 100
MAX_SENTENCE_LENGTH = 400
MIN_SENTENCE_LENGTH = 200
doc_list =[item for item in range(MAX_DATA)]
jump = 3 # jump size for gradient decent
MAX_FEATURE_SIZE = 20 # max size of the gradient descent vector
range_n_clusters = [2, 3, 4, 5, 6] #number of different clustering to compare the performance of the model over different clustering
threshold_cluster_overlapping = 0.7 # to consider two clsuters are similar
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'of', 'within']


In [5]:
# download the dataset as a zip file from the git repo
def dataset_downloader(str_):
  """
  This function download a dataset
  Arguments:
    str_: the name of the dataset. It can be covid or NYT
  Returns:
    df: a pandas dataframe
  """
  if str_ == 'covid':
    url = dataset_NYT_link
  elif str_ == 'NYT':
    url = dataset_covid_link
  else:
    print('dataset is not choose correctly!')

  print('Downloading dataset...')
  res = requests.get(url, allow_redirects=True)
  with open('small_dataset_NYT.csv','wb') as file:
      file.write(res.content)
  print('Download is done!')
  df = pd.read_csv("/content/small_dataset_NYT.csv", header= None, skiprows=1)
  if str_ == 'NYT':
    df.columns = ['id', 'title', 'content', 'publish_time', 'author']
  elif str_ == 'covid':
    df.columns = ['id', 'title', 'content', 'author', 'publish_time']
  
  df = df.dropna(how='any', axis=0)
  print('dataset size after removing non-value cells is {}'.format(df.shape))
  # reduce the size of dataset to dataset_percentage*dataset.shape
  df = df.sample(frac=dataset_percentage)
  print('The new size of dataset is {} and the columns are {}'.format(df.shape, df.columns.values ))
  # remove the content that has less than MIN_SENTENCE_LENGTH words 
  df = df[df['content'].apply(lambda x : len(x.split(' ')))>=MIN_SENTENCE_LENGTH]

  # keep the first 500 words in content
  df['content'] = df['content'].apply(lambda x : ' '.join(x.split(' ')[:MAX_SENTENCE_LENGTH]))
  return df


In [6]:
def special_char_remover(str_):
  """
  This function removes any special characters from a text
  Arguments:
    str_: a string
  Returns:
    A string
  """
  return ' '.join(''.join(w for w in m if w.isalnum()) for m in str_.split(' '))

In [7]:
### create embedding vector for each token in each document
def bert_embedding_generator(df_):
  """
  Generate the embedding for a document
  Arguments:
    df_: a pandas dataframe 
  Returns:
    Embedding_dict: a dictionary of document indices and [embedding vectors, content]
  """
  Embedding_dict = dict()
  print('Start to generate the embedding vector for documents')
  c = 0
  for index, row in df_.iterrows():
    tmp_e = model(tf.constant(tokenizer.encode(row['content']))[None, :])[0][0]
    tmp_d = tokenizer.tokenize(row['content'])
    temp_dict = dict()
    temp_dict['cls'] = tmp_e[0]
    for emb, tok in zip(tmp_e[1:-1], tmp_d):
      temp_dict[tok] = emb
    temp_dict['sep'] = tmp_e[-1]
    Embedding_dict[row['id']] = temp_dict
    c +=1
  print('Embedding is done for {} documents.'.format(c))
  return Embedding_dict

In [8]:
#### Create a list of dataframe out of the dataset. Each dataframe belongs to a timestamp #############
#######################################################################################################
# use 10 days as time interval
def timestamps_generator(df_):
  """
  This function collect data related to each timestamp separately in a dataframe
  Arguments:
    df_: a pandas dataframe
  Returns:
    df_list: a list of all timestamps data [df_1, df_2, ...]
  """
  time_stamp = set()
  df_list= list()
  for index, row in df_.iterrows():
    time_stamp.add(row['publish_time'][0:3])
  for time_interval in time_stamp:
    new_df = df_[df_['publish_time'].str.slice(0, 3)== time_interval]
    if new_df.shape[0]>5: #minimum number of doc in a timestamp
      df_list.append(new_df)
  return df_list

In [9]:
def time_fixer(str_):
  """
  This function convert a date from x/x/xxxx to 0x0xxxxx
  Arguments:
    str_: a date in a string format
  Returns:
    a string without any / and all months and days are in two digit
  """
  t_slot = str_.split('/')
  t_slot[0] = t_slot[0] if len(t_slot[0]) == 2 else '0'+t_slot[0]
  t_slot[1] = t_slot[1] if len(t_slot[1]) == 2 else '0'+t_slot[1]
  return ''+t_slot[0]+t_slot[1]+t_slot[2]

In [10]:
df = dataset_downloader('NYT')
df['content'] = df['content'].apply( lambda x: special_char_remover(x)) # remove special characters
df['publish_time'] = df['publish_time'].apply(lambda x: time_fixer(x))
list_of_dfs = timestamps_generator(df)
Embed_dict = dict() # {timestamp1 {doc1{(number_of_tokens+1, 768)}, doc2{}}, timestamp2{}}
for ts, data_ in zip(range(len(list_of_dfs)), list_of_dfs):
   Embed_dict[ts] = bert_embedding_generator(data_)

# pickle.dump(Embed_dict, open('Embed_dict.p', 'wb')) # store embeddings
# pickle.dump(df, open('list_of_dfs.p', 'wb')) # store list_of_dfs

Download is done!
dataset size after removing non-value cells is (5713, 5)
The new size of dataset is (3428, 5) and the columns are ['id' 'title' 'content' 'publish_time' 'author']
Start to generate the embedding vector for documents


Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors


Embedding is done for 919 documents.
Start to generate the embedding vector for documents
Embedding is done for 16 documents.
Start to generate the embedding vector for documents
Embedding is done for 1172 documents.
Start to generate the embedding vector for documents
Embedding is done for 101 documents.


In [11]:
# Embed_dict = pickle.load(open('Embed_dict.p', 'rb')) #restore the embeddings
# list_of_dfs = pickle.load(open('list_of_dfs.p', 'rb')) #restore the list_of_dfs
# Getting all memory using os.popen()
total_memory, used_memory, free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])
print("RAM memory: used {} GB, total: {} GB".format(used_memory/1000,total_memory/1000))

RAM memory: used 6.92 GB, total: 26.066 GB


In [63]:
def find_similarity(Embed_):
  """
  This function calculate the cosine similarity between each pair of words in a document and sort them as a new vector
  Arguments:
    Embed_: a dictionary of all doc and embedding seperated into their timestamps {timestamp1 {doc1{token1: embedding, token2: embedding}}}
  Returns:
    neighbor_ts_doc: a dictionary of all timestamps, docs and the neighbor similarity for each word
  """
  neighbor_ts_doc = dict()
  for ts_ in Embed_.keys():
    temp = Embed_[ts_]
    final_list = dict()
    for doc in temp.keys():
      neighbor_dict = dict()
      track_dict = dict()
      for f_word, f_emb in temp[doc].items():
        temp_list = list()
        for s_word, s_emb in temp[doc].items():
          if (s_word, f_word) in track_dict.keys():
            temp_list.append(track_dict[(s_word, f_word)])
          else:
            track_dict[(f_word, s_word)] = cosine_similarity([f_emb],[s_emb])[0][0]
            temp_list.append(track_dict[(f_word, s_word)])
        neighbor_dict[f_word] =sorted(temp_list, reverse=True)[1:] # ignore the cosine similarity between a word and itself
      final_list[doc] = neighbor_dict
      print('Document {} is done!'.format(doc))
    neighbor_ts_doc[ts_] = final_list
  return neighbor_ts_doc

In [ ]:
final_vector = find_similarity(Embed_dict)

In [ ]:

#sigmoid function
def amplifier_function(vec_):
  z = array(vec_)
  z = z/np.linalg.norm(z)
  g = 1 / (1 + exp(-z))
  return g

In [ ]:
def gradient_calculator(ns_dict, n_step, MAX_FEATURE_SIZE):
  """
  This function receives all the similarity neighbor for each word in all documents and apply gradient descent over them by the step
  Arguments:
    ns_dict: a dictinary of all neighbor similarity for all tokens
    n_step: the jump value for gradient discent
    MAX_FEATURE_SIZE: the maximum size for each feature victor
  Returns:
    final_dict: a dictionary of all gradient discent of tokens neighbor similarity
  """
  final_dict = dict()
  for ts_ in ns_dict.keys():
    gradient_descent_neighbors = dict()
    word_neighbor_similarity = ns_dict[ts_]
    for doc in word_neighbor_similarity.keys():
      temp_dict = dict()
      for word, embedding in word_neighbor_similarity[doc].items():
        vec_size = len(embedding)
        vec_size -= vec_size%n_step
        embedding = embedding[0:vec_size]
        new_list = [embedding[x:x+n_step] for x in range(0, vec_size, n_step)]
        tmp_list = list()
        for ele in new_list:
          m1,b1 = np.polyfit(np.arange(0, n_step), ele, 1)
          tmp_list.append(m1)
        temp_dict[word] = amplifier_function(tmp_list[0:MAX_FEATURE_SIZE])
      gradient_descent_neighbors[doc] = temp_dict
      print('Document {} is done!'.format(doc))
    final_dict[ts_] = gradient_descent_neighbors
  return final_dict